In [1]:
library(Seurat)
library(SingleR)
library(celldex)
library(pheatmap)
library(dplyr)
library(future)

set.seed(1515)
options(future.globals.maxSize = 80000 * 1024^2)
plan("multicore", workers = 16)
plan()

Attaching SeuratObject

Attaching sp

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderSt

multicore:
- args: function (..., workers = 16, envir = parent.frame())
- tweaked: TRUE
- call: plan("multicore", workers = 16)

In [2]:
harmony <- readRDS("/workdir/wangph/proj/20220418_TB/SC/5.seurat/v7-12sample-normal/2.2.merged-harmony-processed-nodoublet-20220812.rds")

In [3]:
be.se <- BlueprintEncodeData()
be.se
dice.se <- DatabaseImmuneCellExpressionData()
dice.se
hpca.se <- HumanPrimaryCellAtlasData()
hpca.se
mi.se <- MonacoImmuneData()
mi.se
nh.se <- NovershternHematopoieticData()
nh.se

snapshotDate(): 2021-10-19

see ?celldex and browseVignettes('celldex') for documentation

loading from cache

see ?celldex and browseVignettes('celldex') for documentation

loading from cache



class: SummarizedExperiment 
dim: 19859 259 
metadata(0):
assays(1): logcounts
rownames(19859): TSPAN6 TNMD ... LINC00550 GIMAP1-GIMAP5
rowData names(0):
colnames(259): mature.neutrophil
  CD14.positive..CD16.negative.classical.monocyte ...
  epithelial.cell.of.umbilical.artery.1
  dermis.lymphatic.vessel.endothelial.cell.1
colData names(3): label.main label.fine label.ont

snapshotDate(): 2021-10-19

see ?celldex and browseVignettes('celldex') for documentation

loading from cache

see ?celldex and browseVignettes('celldex') for documentation

loading from cache



class: SummarizedExperiment 
dim: 48043 1561 
metadata(0):
assays(1): logcounts
rownames(48043): 5S_rRNA 7SK ... ZZEF1 ZZZ3
rowData names(0):
colnames(1561): TPM_1 TPM_2 ... TPM_101 TPM_102
colData names(3): label.main label.fine label.ont

snapshotDate(): 2021-10-19

see ?celldex and browseVignettes('celldex') for documentation

loading from cache

see ?celldex and browseVignettes('celldex') for documentation

loading from cache



class: SummarizedExperiment 
dim: 19363 713 
metadata(0):
assays(1): logcounts
rownames(19363): A1BG A1BG-AS1 ... ZZEF1 ZZZ3
rowData names(0):
colnames(713): GSM112490 GSM112491 ... GSM92233 GSM92234
colData names(3): label.main label.fine label.ont

snapshotDate(): 2021-10-19

see ?celldex and browseVignettes('celldex') for documentation

loading from cache

see ?celldex and browseVignettes('celldex') for documentation

loading from cache



class: SummarizedExperiment 
dim: 46077 114 
metadata(0):
assays(1): logcounts
rownames(46077): A1BG A1BG-AS1 ... ZYX ZZEF1
rowData names(0):
colnames(114): DZQV_CD8_naive DZQV_CD8_CM ... G4YW_Neutrophils
  G4YW_Basophils
colData names(3): label.main label.fine label.ont

snapshotDate(): 2021-10-19

see ?celldex and browseVignettes('celldex') for documentation

loading from cache

see ?celldex and browseVignettes('celldex') for documentation

loading from cache



class: SummarizedExperiment 
dim: 13276 211 
metadata(0):
assays(1): logcounts
rownames(13276): 13CDNA73 15E1.2 ... ZZEF1 ZZZ3
rowData names(0):
colnames(211): GSM609632 GSM609633 ... GSM609841 GSM609842
colData names(3): label.main label.fine label.ont

In [4]:
#进行singleR注释
aggr_for_SingleR <- GetAssayData(harmony, slot = "data") ##获取标准化矩阵
aggr.singler <- SingleR(test = aggr_for_SingleR, 
                        ref = list(BE = be.se, DICE = dice.se, HPCA = hpca.se, MI = mi.se, NH = nh.se), 
                        labels = list(be.se$label.main, dice.se$label.main, hpca.se$label.main, mi.se$label.main, nh.se$label.main)) 

In [5]:
table(aggr.singler$labels)
table(harmony@meta.data$RNA_snn_res.1)
table(aggr.singler$labels, harmony@meta.data$RNA_snn_res.1)


         Adipocytes          Astrocytes             B cells              B_cell 
               2659                   1                 681                 673 
            B-cells           Basophils        CD4+ T cells        CD4+ T-cells 
              15819                 241                2738               23978 
       CD8+ T cells        CD8+ T-cells        Chondrocytes                 CMP 
                432               35619                 330                   4 
               CMPs                  DC     Dendritic cells   Endothelial cells 
                  7                 630                1424                 326 
  Endothelial_cells    Epithelial cells    Epithelial_cells        Erythrocytes 
                 75                 278                2435                   2 
        Fibroblasts                 GMP                 HSC          HSC_-G-CSF 
               2903                   9                 518                  36 
          HSC_CD34+       K


    0     1    10    11    12    13    14    15    16    17    18    19     2 
18359 13241  4332  4254  3929  3881  3552  2156  2140  2094  1667  1567 11091 
   20    21    22    23    24    25    26    27    28    29     3    30    31 
 1527  1484  1301  1176  1101   871   766   677   607   550  8511   376   360 
   32    33    34     4     5     6     7     8     9 
   99    82    26  7743  5827  5695  5683  5519  5459 

                     
                          0     1    10    11    12    13    14    15    16
  Adipocytes              0     0     0   647     0   843     0     2     0
  Astrocytes              0     0     0     0     0     1     0     0     0
  B cells                 0     0     0     0     0    12     1     8     0
  B_cell                  6     0     0     0     0     0     0     2     1
  B-cells                 6     1     2     5     1    12     0  1753     9
  Basophils               0     0     0     0     1     0     0     0     0
  CD4+ T cells          553    33     0     0    10     4   179     7     0
  CD4+ T-cells        12831  1522     0     2    30    18  2067    52    10
  CD8+ T cells           47   139     0     2     4    11     8     3     0
  CD8+ T-cells         3989 10705     9    29   166   255  1106   119    11
  Chondrocytes            0     0     0   208     0    12     0     0     0
  CMP                     0     0     0     0     0     0     0   

In [6]:
harmony@meta.data$singleR.ori = aggr.singler$labels
head(harmony@meta.data)

,orig.ident,nCount_RNA,nFeature_RNA,sample,library,patient,tissue,pathogen,percent.mt,percent.rb,⋯,RNA_snn_res.0.8,RNA_snn_res.1,RNA_snn_res.1.2,RNA_snn_res.1.5,RNA_snn_res.1.8,RNA_snn_res.2,RNA_snn_res.2.5,RNA_snn_res.3,seurat_clusters,singleR.ori
,<chr>,<dbl>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>
159611-S3_AAACCCAAGAGAAGGT-1,SeuratProject,13578,3127,159611-S3,S3,159611,Lung,TB,3.542495,9.169244,⋯,6,6,13,13,10,11,11,9,9,Macrophages
159611-S3_AAACCCAAGCATAGGC-1,SeuratProject,2379,1220,159611-S3,S3,159611,Lung,TB,1.849517,18.831442,⋯,3,2,2,18,32,21,7,18,18,CD8+ T-cells
159611-S3_AAACCCAAGGCGATAC-1,SeuratProject,5104,1468,159611-S3,S3,159611,Lung,TB,1.900470,28.369906,⋯,7,8,9,33,36,34,36,33,33,Monocytes
159611-S3_AAACCCACACTGCATA-1,SeuratProject,36200,4783,159611-S3,S3,159611,Lung,TB,3.209945,9.475138,⋯,10,10,8,17,15,18,20,21,21,Macrophages
159611-S3_AAACCCACATCCTGTC-1,SeuratProject,3813,1461,159611-S3,S3,159611,Lung,TB,3.829006,1.022817,⋯,27,22,34,38,41,43,48,57,57,Monocytes
159611-S3_AAACCCACATTGTGCA-1,SeuratProject,3373,1368,159611-S3,S3,159611,Lung,TB,1.126594,23.450934,⋯,3,2,2,3,3,2,1,2,2,CD8+ T-cells


In [7]:
saveRDS(harmony, file = "/workdir/wangph/proj/20220418_TB/SC/5.seurat/v7-12sample-normal/3.2.merged-harmony-processed-singleR-20220812.rds")